# Algorithm & Metric Deep Dive - Making appropriate choices for your problem

## Overview 

In the previous notebook, we built a machine learning containing all the important elements we would find in any sophisticated, real world pipeline, but in our example we used quite simple components for each step. Even still, there were quite a number of *hyperparameters* we had to choose along the way, without having understood what we were choosing and why, or more importantly, what would be a good choice for our particular problem and the dataset. This notebook will attempt to give a little insight into how key ML algorithms work with aim of giving some basic understanding of what the hyperparameters mean and how they influence the end result. This will hopefully be a starting point for making approproate chopices of algorithm, hyperparamters and metrics. 


### Prerequisites
* Completed notebooks 1 & 2

### Learning Outcomes 

* Understand mechanisms of key tree based and neural network algorithms
* Understand key hyperparameters and how to choose them
* Understand key metrics and how to select the right one for your problem

### Links to Best practices and Values
* ML Pitfalls - Avoid problems such as overfitting and underfitting through appropriate choice of algorithms and hyperparameters
* Ethics - Be able to justify your choices for what you have implemented.
* ML Lifecycle - Ensure you are able to reproduce results


### Data Science Framework
* Weather regimes - Discovery and Attribution
* Radiation emulation - Fusing Simulation and Data Science
* XBT - Uncertainty and Trust
* Rotors - Data to Decisions

## Tutorial - Decision Trees
Decision trees


Root node: The base of the decision tree.
Splitting: The process of dividing a node into multiple sub-nodes.
Decision node: When a sub-node is further split into additional sub-nodes.
Leaf node: When a sub-node does not further split into additional sub-nodes; represents possible outcomes.
Pruning: The process of removing sub-nodes of a decision tree.
Branch: A subsection of the decision tree consisting of multiple nodes.
Hyoerparameters
Max depth
Image of tree https://miro.medium.com/max/1400/1*3P1333UmqEww6YMpjisj4Q.png from article https://towardsdatascience.com/decision-trees-explained-3ec41632ceb6 

Maths of CART algorithm

https://towardsdatascience.com/the-mathematics-of-decision-trees-random-forest-and-feature-importance-in-scikit-learn-and-spark-f2861df67e3
How does this relate to decision trees hyperparameters

In the decision tree, the nodes are split into subnodes on the basis of a threshold value of an attribute. The CART algorithm does that by searching for the best homogeneity for the subnodes, with the help of the Gini Index criterion. 
Quote from https://www.analyticssteps.com/blogs/classification-and-regression-tree-cart-algorithm  
Training terms
* Greedy algorithm
* Stopping criteria
* Pruning

Use XBT example for trees
Show XBT flowchart
Example code
Régression va classification trees

Décision tree visualisation
Variants:
• random forest
• Gradient boosted
• Xgboost 
Random forest discussion 
Deals with variance Leo breiman
Key terms
Bgging
Bootstapping
aggregation
Ensemble model


Advantages

Works for numerical or categorical data and variables.
Models problems with multiple outputs.
Tests the reliability of the tree.
Requires less data cleaning than other data modeling techniques. Easy to explain to those without an analytical background.

Disadvantages
Affected by noise in the data.
Not ideal for large datasets.
Can disproportionately value, or weigh, attributes.
The decisions at nodes are limited to binary outcomes, reducing the complexity that the tree can handle. Trees can become very complex when dealing with uncertainty and numerous linked outcomes. 




# Example: Decision Trees - XBT classification

In [ ]:
# load in xbt data
# clean xbt data

In [ ]:
# preprocess
# select features
# prepare data

In [ ]:
# setup and train decision tree

In [ ]:
# visualise branches of decision tree

In [ ]:
# train random forest


In [4]:
# look at internal of random forest (number of trees, bags etc.)

In [ ]:
# evaluate results from random forest and decision tree

## Tutorial - Neural Networks

Description of Neural networks
* Explain a single perceptron
* History
* Weighted sum plus threshold (binary activation)
* Non linear activation, sigmoid neuron

Key terms
* neuron
* Perceptron
* Activation
* Weights
* Bias - the constant term
* Sigmoid function

Multi layer
* How are they joined together?

How do we train? 
* Gradient descent
* Stochastic gradient decscent
* Back propagation 

Key terms
* gradient descent
* Learning rate
* back propagation
* mini batch
* epoch
* copst function

Hyperparameters
* batch size
* learning rate
* solver
* maximum iterations

 
Types of NN
* feed forward
* Convolutional Neural Network
* Recurrent Neural Network
* Graphical Neural Network
 

## Example - Scorates Radiation Model Emulation

Explain problem and dataset

In [ ]:
# load in scorates data

In [ ]:
# prepreprroces

In [ ]:
# train test split??

In [2]:
# construct FF architecture

In [ ]:
# train NN

In [ ]:
# evaluate

In [ ]:
# train 1D CNN

In [ ]:
# evaluate and compare

In [ ]:
# load rotors data

In [ ]:
# clean, preprocess, split based on year

In [ ]:
# create LSTM

In [ ]:
# train LSTM

In [3]:
# evaluate (compare to previous notebook results)

## Tutorial - Metrics


Description of metrics
* classification metrics
* regression metrics

## Excercise - Metrics
xx


## Examples of use
xx


## Next steps
Rotor Challenge
Radiation emulation




## Dataset Info
* XBT Data
* Radiation Emulation
* Rotors

## References


Decision trees
* https://www.mastersindatascience.org/learning/introduction-to-machine-learning-algorithms/decision-tree/#:~:text=A
* https://towardsdatascience.com/decision-trees-explained-3ec41632ceb6
* 

Neural Networks
* [Introduction to Neural Networks - Kaggle](https://www.kaggle.com/code/carlosaguayo/introduction-to-neural-networks/notebook)
* [Back propagation - wikipedia](https://en.wikipedia.org/wiki/Backpropagation)
*[Back propagation - brilliant wiki](https://brilliant.org/wiki/backpropagation/#:~:text=Backpropagation%2C%20short%20for%20%22backward%20propagation,to%20the%20neural%20network's%20weights)
* [Introduction to Deep Learning - Kaggle](https://www.kaggle.com/learn/intro-to-deep-learning)
* [Introduction to Neural Networks - IBM](https://www.ibm.com/cloud/learn/neural-networks)
* [Neural Networks - MIT](https://news.mit.edu/2017/explained-neural-networks-deep-learning-0414)

* RNN
* CNN
* GNN

Metrics
* [Regression and Classification metrics - scikit-learn](https://scikit-learn.org/stable/modules/model_evaluation.html)
* 

